In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from unidecode import unidecode

In [27]:
#FatosGeração (ID empreendiemnto) > fatodadosmetereologicos (vento, precip, etc) > dimempreendimentos (tipogeração)

#dim_empreendimentos -> id;NomEmpreendimento;SigUFPrincipal;SigTipoGeracao;NomFonteCombustivel
#fato_dados_metereologicos -> id;Data;Hora UTC;precipTotalHorario;ventoVeloHoraria;tempAr;radGlobal;umidRelAr;id_regiao
#fato_geracao -> id;DatGeracaoConjuntoDados;MdaPotenciaFiscalizadaKw;MdaPotenciaOutorgadaKw;Id_empreendimento
#id;UF;latitude;longitude;altitude
#Pegar empreendimento (id) > Pegar todos os dados meterologicos (preciptação, vento, temp...) > Pegar qual o TipoGeração na região

dim_regiao = pd.read_csv(
    "data/dim_regiao.csv",
    delimiter=";",
    encoding="UTF-8")

dim_empreendimentos = pd.read_csv(
    "data/dim_empreendimento.csv",
    delimiter=";",
    encoding="UTF-8")

fato_dados_metereologicos = pd.read_csv(
    "data/fato_dados_meteorologicos.csv",
    delimiter=";",
    encoding="UTF-8")

fato_geracao = pd.read_csv(
    "data/fato_geracao.csv",
    delimiter=";",
    encoding="UTF-8",
    decimal=',')

#sklearn.ensemble.HistGradientBoostingClassifier


In [28]:
# dim_empreendimentos['DscMuninicpios'].str.split('-')[0][0].strip()

dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].apply(lambda x: unidecode(str(x).split('-')[0].strip().upper()))
dim_empreendimentos['DscMuninicpios']

0               NOVA LIMA
1               NOVA LIMA
2               NOVA LIMA
3         BENTO GONCALVES
4        FLORIANO PEIXOTO
               ...       
24595              MANAUS
24596              MANAUS
24597              GOIANA
24598             GOIOERE
24599           CARIACICA
Name: DscMuninicpios, Length: 24600, dtype: object

In [29]:
#dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].str.split('-', 1).str[0]
# dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].str.split('-')[0][0].strip()
# dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].str.upper()
empreendimentos_combinados = pd.merge(fato_geracao, dim_empreendimentos, left_on='Id_empreendimento', right_on='id')
#dim_empreendimentos.head()







empreendimentos_combinados = empreendimentos_combinados.drop('DatGeracaoConjuntoDados', axis=1)

empreendimentos_combinados.head()


,id_x,MdaPotenciaFiscalizadaKw,MdaPotenciaOutorgadaKw,Id_empreendimento,id_y,NomEmpreendimento,SigUFPrincipal,SigTipoGeracao,NomFonteCombustivel,DscMuninicpios
0,0,1400,1400.0,0,0,E,MG,PCH,Potencial hidráulico,NOVA LIMA
1,1,3972,3972.0,1,1,F,MG,PCH,Potencial hidráulico,NOVA LIMA
2,2,1440,1440.0,2,2,G,MG,PCH,Potencial hidráulico,NOVA LIMA
3,3,100000,100000.0,3,3,14 de Julho,RS,UHE,Potencial hidráulico,BENTO GONCALVES
4,4,720,720.0,4,4,Abaúna,RS,CGH,Potencial hidráulico,FLORIANO PEIXOTO


In [30]:
dim_regiao.head()

,id,UF,cidade,latitude,longitude,altitude
0,0,MT,PARANATINGA,"-14,42138888","-54,0361111","476,91"
1,1,BA,GUANAMBI,"-14,20805555","-42,74972221","552,23"
2,2,CE,CRATEUS,"-5,18666666","-40,67222221","298,19"
3,3,PA,PLACAS,"-3,86416666","-54,21638888","100,45"
4,4,MG,BARBACENA,"-21,22833332","-43,76777777","1168,76"


In [31]:
#Cidade, MediaPrecipTotalHorarioCidade, MediaVentoVeloHorariaCidade, MediaTempArCidade, ...., QuantidadeEnergiaEnergiaSolarGeradaPorCidade, QuantidadeEnergiaNuclearGeradaPorCidade, QuantidadeEnergiaTermalGeradaPorCidade, .....

dados_combinados = pd.merge(fato_dados_metereologicos, dim_regiao[['id', 'UF','cidade']], left_on='id_regiao', right_on='id')

#Tratando dados decimais para virarem tipo float
#dados_combinados = pd.merge(fato_dados_metereologicos, dim_regiao, left_on='id_regiao', right_on='id')

dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].astype(str)
dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].str.replace(',', '.')
dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].astype(float)

dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].astype(str)
dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].str.replace(',', '.')
dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].astype(float)

dados_combinados['tempAr'] = dados_combinados['tempAr'].astype(str)
dados_combinados['tempAr'] = dados_combinados['tempAr'].str.replace(',', '.')
dados_combinados['tempAr'] = dados_combinados['tempAr'].astype(float)

dados_combinados['radGlobal'] = dados_combinados['radGlobal'].astype(str)
dados_combinados['radGlobal'] = dados_combinados['radGlobal'].str.replace(',', '.')
dados_combinados['radGlobal'] = dados_combinados['radGlobal'].astype(float)

In [32]:
siglasUF = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']


#dados_combinados.head()

#dados_combinados['cidade']
sample_dados_combinados = dados_combinados.sample(frac=0.1, random_state=42)
sample_dados_combinados

,id_x,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao,timestamp,id_y,UF,cidade
915995,915995,0.029738,4.500000,25.4,231.900000,0.0,147,2022-07-14 20:00:00,147,PB,CAMARATUBA
859452,859452,0.000000,1.300000,20.2,1371.112837,83.0,136,2022-08-23 23:00:00,136,ES,VILA VELHA
1804653,1804653,0.000000,2.239241,23.5,1404.817852,81.0,299,2022-02-03 02:00:00,299,RO,VILHENA
1908538,1908538,9.800000,1.500000,20.4,1452.793701,91.0,316,2022-03-13 00:00:00,316,MG,UBERLANDIA
3163682,3163682,0.000000,0.900000,20.3,957.632783,72.0,517,2022-07-19 02:00:00,517,MT,CAMPO NOVO DOS PARECIS
...,...,...,...,...,...,...,...,...,...,...,...
3068208,3068208,0.000000,0.700000,19.4,1461.322303,78.0,500,2022-09-29 05:00:00,500,DF,GAMA (PONTE ALTA)
2624005,2624005,0.000000,1.200000,23.4,1459.568543,82.0,429,2022-09-23 07:00:00,429,PA,CONCEICAO DO ARAGUAIA
1404208,1404208,0.000000,0.000000,31.1,1422.700000,34.0,236,2022-09-10 13:00:00,236,MS,PARANAIBA
1289597,1289597,0.000000,0.000000,0.0,1739.609085,0.0,215,2022-09-14 01:00:00,215,AL,PAO DE ACUCAR


In [33]:
df_final = pd.merge(sample_dados_combinados, empreendimentos_combinados, left_on='cidade', right_on='DscMuninicpios')


df_final

,id_x_x,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao,timestamp,id_y_x,UF,...,id_x_y,MdaPotenciaFiscalizadaKw,MdaPotenciaOutorgadaKw,Id_empreendimento,id_y_y,NomEmpreendimento,SigUFPrincipal,SigTipoGeracao,NomFonteCombustivel,DscMuninicpios
0,859452,0.0,1.3,20.2,1371.112837,83.0,136,2022-08-23 23:00:00,136,ES,...,910,3646,3646.0,910,910,Praia da Costa,ES,UTE,Gás Natural,VILA VELHA
1,859452,0.0,1.3,20.2,1371.112837,83.0,136,2022-08-23 23:00:00,136,ES,...,2005,1800,1800.0,2005,2005,Shopping Praia da Costa,ES,UTE,Óleo Diesel,VILA VELHA
2,859452,0.0,1.3,20.2,1371.112837,83.0,136,2022-08-23 23:00:00,136,ES,...,2473,347,347.0,2473,2473,Auto Serviço Faé,ES,UTE,Óleo Diesel,VILA VELHA
3,859452,0.0,1.3,20.2,1371.112837,83.0,136,2022-08-23 23:00:00,136,ES,...,3867,760,760.0,3867,3867,Shopping Vila Velha,ES,UTE,Óleo Diesel,VILA VELHA
4,859452,0.0,1.3,20.2,1371.112837,83.0,136,2022-08-23 23:00:00,136,ES,...,4351,800,800.0,4351,4351,Atacadão SA Vila Velha,ES,UTE,Óleo Diesel,VILA VELHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4442105,2687585,0.0,0.6,23.9,48.100000,93.0,441,2022-04-11 10:00:00,441,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES
4442106,2687569,0.0,2.4,30.7,2613.500000,63.0,441,2022-04-10 18:00:00,441,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES
4442107,2687680,0.0,0.2,24.1,0.000000,94.0,441,2022-04-15 09:00:00,441,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES
4442108,2687617,1.2,0.6,27.9,765.000000,88.0,441,2022-04-12 18:00:00,441,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES


In [34]:
colunas_com_nan = df_final.columns[df_final.isnull().any()].tolist()
print("Colunas com valores NaN:")
for coluna in colunas_com_nan:
    print(coluna)

Colunas com valores NaN:
